In [18]:


import sys
import os
import hashlib
import struct
import subprocess
import collections
import tensorflow as tf
from tensorflow.core.example import example_pb2
import nltk
import pandas as pd

import re
from nltk.corpus import stopwords

In [45]:
data = pd.read_csv('G:\\master thesis\\bigPatent_dataset_csv\\train\\val_pgn_E.csv')
##data['content'][0]

In [46]:
data['content'][0]

'the invention comprises a pair of tubes with a first tube having a top and bottom . the bottom of the first tube comprises an open bottom and has an interior profile , the preferred profile is frusto - conical . the second tube having a top and bottom . the bottom of the second tube having a preferred exterior profile that is adapted to mate with the interior profile of the first tube , thereby sealing the open bottom of the first tube when the profiles come into contact with each other . open slots are provided for in the profile of the second tube to allow transfer of material from the interior of the second tube out through the open bottom of the first tube when the profiles disengage . a more specific detail of the preferred embodiment is further discussed below . the preferred embodiment 100 comprises an exterior casing 6 and an interior mandrel 5 . see , e . g , fig1 . the casing 6 and mandrel 5 may be of any cross section so long as mandrel 5 is able to fit slidably within casi

In [47]:
data['title'][0]

'the present invention is an apparatus and a method for casting a cementitious or stone pile into the ground . the present invention comprises an exterior driving casing and an interior mandrel acting in cooperation to hold and deliver the pile forming material . a retrievable driving driving shoe is also disclose .'

In [40]:
data.shape[0]

34443

In [23]:
contractions =          {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have", "i've": "i have"}

In [24]:


#for cleaning text 
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    ##text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    ##text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    ##text = re.sub(r'<br />', ' ', text)
    ##text = re.sub(r'\'', ' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

In [26]:
dm_single_close_quote = u'\u2019' # unicode
dm_double_close_quote = u'\u201d'
END_TOKENS = ['.', '!', '?', '...', "'", "`", '"', dm_single_close_quote, dm_double_close_quote, ")"] # acceptable ways to end a sentence

# We use these to separate the summary sentences in the .bin datafiles
SENTENCE_START = '<s>'
SENTENCE_END = '</s>'

all_train_urls = ""
all_val_urls = ""
all_test_urls = ""

cnn_tokenized_stories_dir_train = "G:\\master thesis\\bigPatent_dataset_csv\\train\\cnn_stories_tokenized_train" #location of folder to tokenize text
cnn_tokenized_stories_dir_test = "G:\\master thesis\\bigPatent_dataset_csv\\train\\cnn_stories_tokenized_test" #location of folder to tokenize text
cnn_tokenized_stories_dir_val = "G:\\master thesis\\bigPatent_dataset_csv\\train\\cnn_stories_tokenized_val" #location of folder to tokenize text
dm_tokenized_stories_dir = "G:\\master thesis\\bigPatent_dataset_csv\\train\\dm_stories_tokenized" #not used
finished_files_dir = "G:\\master thesis\\bigPatent_dataset_csv\\train\\bp_finished_files" #final ouput
chunks_dir = os.path.join(finished_files_dir, "chunked")



VOCAB_SIZE = 200000
CHUNK_SIZE = 1000 # num examples per chunk, for the chunked data

In [27]:
def chunk_file(set_name):
    
      in_file = finished_files_dir + '/%s.bin' % set_name
      reader = open(in_file, "rb")
      chunk = 0
      finished = False
      while not finished:
        chunk_fname = os.path.join(chunks_dir, '%s_%03d.bin' % (set_name, chunk)) # new chunk
        with open(chunk_fname, 'wb') as writer:
          for _ in range(CHUNK_SIZE):
            len_bytes = reader.read(8)
            if not len_bytes:
              finished = True
              break
            str_len = struct.unpack('q', len_bytes)[0]
            example_str = struct.unpack('%ds' % str_len, reader.read(str_len))[0]
            writer.write(struct.pack('q', str_len))
            writer.write(struct.pack('%ds' % str_len, example_str))
          chunk += 1

In [28]:
def chunk_all():
  # Make a dir to hold the chunks
  if not os.path.isdir(chunks_dir):
    os.mkdir(chunks_dir)
  # Chunk the data
  for set_name in ['train', 'val', 'test']:
    print ("Splitting %s data into chunks..." % set_name)
    chunk_file(set_name)
  print ("Saved chunked data in %s" % chunks_dir)

In [29]:
def tokenize_stories(reviews, tokenized_stories_dir):
  """Maps a whole directory of .story files to a tokenized version using Stanford CoreNLP Tokenizer"""
  #progress = ProgressBar.ProgressBar(len(reviews), fmt=ProgressBar.ProgressBar.FULL)
  print('open progress bar')

  for i, row in reviews.iterrows():
        #if i==20:
        #    break
        filename = str(i) + '.tok'
        with open(os.path.join(tokenized_stories_dir, filename), 'w', encoding="utf-8") as temp_file:
            text = row["content"]
            text = clean_text(text , remove_stopwords = True)
            #tok = nltk.word_tokenize(text) #default
            tok = nltk.sent_tokenize(text) #abdullah modified
            tok.append("@highlight")
            Summary = row["title"]
            Summary = clean_text(Summary ,remove_stopwords = False)
            #tok.extend(nltk.word_tokenize(Summary)) #default
            tok.extend(nltk.sent_tokenize(Summary)) # abdullah modified
            list = tok.copy()

            for i in tok:
                if(i=='``' or i=="''" ):
                    list.remove(i)
            tok_string = "\n".join(str(x) for x in list)
            temp_file.write(tok_string)

        #progress.current += 1
        #progress()
  print ("Successfully finished tokenizing to %s .\n" % (tokenized_stories_dir))

In [30]:
def fix_missing_period(line):
  """Adds a period to a line that is missing a period"""
  if "@highlight" in line: return line
  if line=="": return line
  if line[-1] in END_TOKENS: return line
  # print line[-1]
  return line + " ." # abdullah mofidied # default return line + " ."


def read_text_file(text_file):
  lines = []
  with open(text_file, "r", encoding="utf-8") as f:
    for line in f:
      lines.append(line.strip())
  return lines

In [31]:
def get_art_abs(story_file):
  lines = read_text_file(story_file)

  # Lowercase everything
  lines = [line.lower() for line in lines]

  # Put periods on the ends of lines that are missing them (this is a problem in the dataset because many image captions don't end in periods; consequently they end up in the body of the article as run-on sentences)
  lines = [fix_missing_period(line) for line in lines]

  # Separate out article and abstract sentences
  article_lines = []
  highlights = []
  next_is_highlight = False
  for idx,line in enumerate(lines):
    if line == "":
      continue # empty line
    elif line.startswith("@highlight"):
      next_is_highlight = True 
    elif next_is_highlight: 
      highlights.append(line)
    else:
      article_lines.append(line)

  # Make article into a single string
  article = ' '.join(article_lines)

  # Make abstract into a signle string, putting <s> and </s> tags around the sentences
  abstract = ' '.join(["%s %s %s" % (SENTENCE_START, sent, SENTENCE_END) for sent in highlights])

  return article, abstract

In [32]:

def write_to_bin_train(file_names, out_file, makevocab=False):
  """Reads the tokenized .story files corresponding to the urls listed in the url_file and writes them to a out_file."""
 
  story_fnames = [str(s)+".tok" for s in file_names]
  num_stories = len(story_fnames)

  if makevocab:
    vocab_counter = collections.Counter()

  with open(out_file, 'wb') as writer:
    for idx,s in enumerate(story_fnames):
      if idx % 1000 == 0:
        print( "Writing story %i of %i; %.2f percent done" % (idx, num_stories, float(idx)*100.0/float(num_stories)))

      # Look in the tokenized story dirs to find the .story file corresponding to this url
      if os.path.isfile(os.path.join(cnn_tokenized_stories_dir_train, s)):
        story_file = os.path.join(cnn_tokenized_stories_dir_train, s)
      elif os.path.isfile(os.path.join(dm_tokenized_stories_dir, s)):
        story_file = os.path.join(dm_tokenized_stories_dir, s)
      else:
        print ("Error: Couldn't find tokenized story file %s in either tokenized story directories %s and %s. Was there an error during tokenization?" % (s, cnn_tokenized_stories_dir, dm_tokenized_stories_dir))
        # Check again if tokenized stories directories contain correct number of files
        print ("Checking that the tokenized stories directories %s and %s contain correct number of files..." % (cnn_tokenized_stories_dir, dm_tokenized_stories_dir))
        #check_num_stories(cnn_tokenized_stories_dir, num_expected_cnn_stories)
        #check_num_stories(dm_tokenized_stories_dir, num_expected_dm_stories)
        #raise Exception("Tokenized stories directories %s and %s contain correct number of files but story file %s found in neither." % (cnn_tokenized_stories_dir, dm_tokenized_stories_dir, s))
        
      # Get the strings to write to .bin file
      article, abstract = get_art_abs(story_file)

      
      # Write to tf.Example
      tf_example = example_pb2.Example()
      tf_example.features.feature['article'].bytes_list.value.extend([article.encode('utf-8')])
      tf_example.features.feature['abstract'].bytes_list.value.extend([abstract.encode('utf-8')])
      tf_example_str = tf_example.SerializeToString()
      str_len = len(tf_example_str)
      writer.write(struct.pack('q', str_len))
      writer.write(struct.pack('%ds' % str_len, tf_example_str))
   

      # Write the vocab to file, if applicable
      if makevocab:
        art_tokens = article.split(' ')
        abs_tokens = abstract.split(' ')
        abs_tokens = [t for t in abs_tokens if t not in [SENTENCE_START, SENTENCE_END]] # remove these tags from vocab
        tokens = art_tokens + abs_tokens
        tokens = [t.strip() for t in tokens] # strip
        tokens = [t for t in tokens if t!=""] # remove empty
        vocab_counter.update(tokens)

  print ("Finished writing file %s\n" % out_file)

  # write vocab to file
  if makevocab:
    print ("Writing vocab file...")
    with open(os.path.join(finished_files_dir, "vocab"), 'w', encoding="utf-8") as writer:
      for word, count in vocab_counter.most_common(VOCAB_SIZE):
        writer.write(word + ' ' + str(count) + '\n')
    print ("Finished writing vocab file")

In [33]:

def write_to_bin_test(file_names, out_file, makevocab=False):
  """Reads the tokenized .story files corresponding to the urls listed in the url_file and writes them to a out_file."""
 
  story_fnames = [str(s)+".tok" for s in file_names]
  num_stories = len(story_fnames)

  if makevocab:
    vocab_counter = collections.Counter()

  with open(out_file, 'wb') as writer:
    for idx,s in enumerate(story_fnames):
      if idx % 1000 == 0:
        print( "Writing story %i of %i; %.2f percent done" % (idx, num_stories, float(idx)*100.0/float(num_stories)))

      # Look in the tokenized story dirs to find the .story file corresponding to this url
      if os.path.isfile(os.path.join(cnn_tokenized_stories_dir_test, s)):
        story_file = os.path.join(cnn_tokenized_stories_dir_test, s)
      elif os.path.isfile(os.path.join(dm_tokenized_stories_dir, s)):
        story_file = os.path.join(dm_tokenized_stories_dir, s)
      else:
        print ("Error: Couldn't find tokenized story file %s in either tokenized story directories %s and %s. Was there an error during tokenization?" % (s, cnn_tokenized_stories_dir, dm_tokenized_stories_dir))
        # Check again if tokenized stories directories contain correct number of files
        print ("Checking that the tokenized stories directories %s and %s contain correct number of files..." % (cnn_tokenized_stories_dir, dm_tokenized_stories_dir))
        #check_num_stories(cnn_tokenized_stories_dir, num_expected_cnn_stories)
        #check_num_stories(dm_tokenized_stories_dir, num_expected_dm_stories)
        #raise Exception("Tokenized stories directories %s and %s contain correct number of files but story file %s found in neither." % (cnn_tokenized_stories_dir, dm_tokenized_stories_dir, s))
        
      # Get the strings to write to .bin file
      article, abstract = get_art_abs(story_file)

      
      # Write to tf.Example
      tf_example = example_pb2.Example()
      tf_example.features.feature['article'].bytes_list.value.extend([article.encode('utf-8')])
      tf_example.features.feature['abstract'].bytes_list.value.extend([abstract.encode('utf-8')])
      tf_example_str = tf_example.SerializeToString()
      str_len = len(tf_example_str)
      writer.write(struct.pack('q', str_len))
      writer.write(struct.pack('%ds' % str_len, tf_example_str))
   

      # Write the vocab to file, if applicable
      if makevocab:
        art_tokens = article.split(' ')
        abs_tokens = abstract.split(' ')
        abs_tokens = [t for t in abs_tokens if t not in [SENTENCE_START, SENTENCE_END]] # remove these tags from vocab
        tokens = art_tokens + abs_tokens
        tokens = [t.strip() for t in tokens] # strip
        tokens = [t for t in tokens if t!=""] # remove empty
        vocab_counter.update(tokens)

  print ("Finished writing file %s\n" % out_file)

  # write vocab to file
  if makevocab:
    print ("Writing vocab file...")
    with open(os.path.join(finished_files_dir, "vocab"), 'w', encoding="utf-8") as writer:
      for word, count in vocab_counter.most_common(VOCAB_SIZE):
        writer.write(word + ' ' + str(count) + '\n')
    print ("Finished writing vocab file")

In [34]:

def write_to_bin_val(file_names, out_file, makevocab=False):
  """Reads the tokenized .story files corresponding to the urls listed in the url_file and writes them to a out_file."""
 
  story_fnames = [str(s)+".tok" for s in file_names]
  num_stories = len(story_fnames)

  if makevocab:
    vocab_counter = collections.Counter()

  with open(out_file, 'wb') as writer:
    for idx,s in enumerate(story_fnames):
      if idx % 1000 == 0:
        print( "Writing story %i of %i; %.2f percent done" % (idx, num_stories, float(idx)*100.0/float(num_stories)))

      # Look in the tokenized story dirs to find the .story file corresponding to this url
      if os.path.isfile(os.path.join(cnn_tokenized_stories_dir_val, s)):
        story_file = os.path.join(cnn_tokenized_stories_dir_val, s)
      elif os.path.isfile(os.path.join(dm_tokenized_stories_dir, s)):
        story_file = os.path.join(dm_tokenized_stories_dir, s)
      else:
        print ("Error: Couldn't find tokenized story file %s in either tokenized story directories %s and %s. Was there an error during tokenization?" % (s, cnn_tokenized_stories_dir, dm_tokenized_stories_dir))
        # Check again if tokenized stories directories contain correct number of files
        print ("Checking that the tokenized stories directories %s and %s contain correct number of files..." % (cnn_tokenized_stories_dir, dm_tokenized_stories_dir))
        #check_num_stories(cnn_tokenized_stories_dir, num_expected_cnn_stories)
        #check_num_stories(dm_tokenized_stories_dir, num_expected_dm_stories)
        #raise Exception("Tokenized stories directories %s and %s contain correct number of files but story file %s found in neither." % (cnn_tokenized_stories_dir, dm_tokenized_stories_dir, s))
        
      # Get the strings to write to .bin file
      article, abstract = get_art_abs(story_file)

      
      # Write to tf.Example
      tf_example = example_pb2.Example()
      tf_example.features.feature['article'].bytes_list.value.extend([article.encode('utf-8')])
      tf_example.features.feature['abstract'].bytes_list.value.extend([abstract.encode('utf-8')])
      tf_example_str = tf_example.SerializeToString()
      str_len = len(tf_example_str)
      writer.write(struct.pack('q', str_len))
      writer.write(struct.pack('%ds' % str_len, tf_example_str))
   

      # Write the vocab to file, if applicable
      if makevocab:
        art_tokens = article.split(' ')
        abs_tokens = abstract.split(' ')
        abs_tokens = [t for t in abs_tokens if t not in [SENTENCE_START, SENTENCE_END]] # remove these tags from vocab
        tokens = art_tokens + abs_tokens
        tokens = [t.strip() for t in tokens] # strip
        tokens = [t for t in tokens if t!=""] # remove empty
        vocab_counter.update(tokens)

  print ("Finished writing file %s\n" % out_file)

  # write vocab to file
  if makevocab:
    print ("Writing vocab file...")
    with open(os.path.join(finished_files_dir, "vocab"), 'w', encoding="utf-8") as writer:
      for word, count in vocab_counter.most_common(VOCAB_SIZE):
        writer.write(word + ' ' + str(count) + '\n')
    print ("Finished writing vocab file")

In [36]:
def check_num_stories(stories_dir, num_expected):
  num_stories = len(os.listdir(stories_dir))
  if num_stories != num_expected:
    raise Exception("stories directory %s contains %i files but should contain %i" % (stories_dir, num_stories, num_expected))

In [37]:

if __name__ == '__main__':
  #main directory
  cnn_stories_dir =  r"G:\\master thesis\\bigPatent_dataset_csv\\train"

  ####### modified by abdullah (for training data) ################
  # Create some new directories
  if not os.path.exists(cnn_tokenized_stories_dir_train): os.makedirs(cnn_tokenized_stories_dir_train)
  if not os.path.exists(finished_files_dir): os.makedirs(finished_files_dir)

  #data needed is in a csv format
  #containg 2 columbs (content , title)
  reviews_csv =cnn_stories_dir + "\\tr_pgn_E.csv"
  reviews_tr = pd.read_csv(reviews_csv)
  reviews_tr = reviews_tr.filter(['content', 'title'])
  reviews_tr = reviews_tr.dropna()
  reviews_tr = reviews_tr.reset_index(drop=True)
  reviews_tr.head()
    
    
    

  # Run nltk tokenizer on both text and summary , outputting to tokenized stories directories
  tokenize_stories(reviews_tr, cnn_tokenized_stories_dir_train)

  #to get the length of your dataset
  num_expected_cnn_stories_tr =reviews_tr.shape[0]
    
    
    
   ####### modified by abdullah (for training data) ################
  # Create some new directories
  if not os.path.exists(cnn_tokenized_stories_dir_test): os.makedirs(cnn_tokenized_stories_dir_test)
  #if not os.path.exists(finished_files_dir): os.makedirs(finished_files_dir)

  #data needed is in a csv format
  #containg 2 columbs (content , title)
  reviews_csv =cnn_stories_dir + "\\ts_pgn_E.csv"
  reviews_ts = pd.read_csv(reviews_csv)
  reviews_ts = reviews_ts.filter(['content', 'title'])
  reviews_ts = reviews_ts.dropna()
  reviews_ts = reviews_ts.reset_index(drop=True)
  reviews_ts.head()


# Run nltk tokenizer on both text and summary , outputting to tokenized stories directories
  tokenize_stories(reviews_ts, cnn_tokenized_stories_dir_test)

  #to get the length of your dataset
  num_expected_cnn_stories_ts =reviews_ts.shape[0]

    
    
  ####### modified by abdullah (for training data) ################
  # Create some new directories
  if not os.path.exists(cnn_tokenized_stories_dir_val): os.makedirs(cnn_tokenized_stories_dir_val)
  #if not os.path.exists(finished_files_dir): os.makedirs(finished_files_dir)

  #data needed is in a csv format
  #containg 2 columbs (content , title)
  reviews_csv =cnn_stories_dir + "\\val_pgn_E.csv"
  reviews_val = pd.read_csv(reviews_csv)
  reviews_val = reviews_val.filter(['content', 'title'])
  reviews_val = reviews_val.dropna()
  reviews_val = reviews_val.reset_index(drop=True)
  reviews_val.head()  
    
    
    

  # Run nltk tokenizer on both text and summary , outputting to tokenized stories directories
  tokenize_stories(reviews_val, cnn_tokenized_stories_dir_val)

  #to get the length of your dataset
  num_expected_cnn_stories_val =reviews_val.shape[0]  
    
    
    

  #testing len = 2000
  #validation lenght = 2000
  ##all_train_urls = range(0,num_expected_cnn_stories-2000)
  ##all_val_urls = range(num_expected_cnn_stories-2000,num_expected_cnn_stories-1000)
  ##all_test_urls = range(num_expected_cnn_stories-1000,num_expected_cnn_stories)

  all_train_urls = range(0,num_expected_cnn_stories_tr)
  all_val_urls = range(0,num_expected_cnn_stories_val)
  all_test_urls = range(0,num_expected_cnn_stories_ts)

  #for testing
  ##############all_train_urls= range(0,80)
  ##############all_val_urls = range(80,90)
  ##############all_test_urls = range(90,100)

  # Read the tokenized stories, do a little postprocessing then write to bin files
  write_to_bin_test(all_test_urls, os.path.join(finished_files_dir, "test.bin"))
  write_to_bin_val(all_val_urls, os.path.join(finished_files_dir, "val.bin"))
  write_to_bin_train(all_train_urls, os.path.join(finished_files_dir, "train.bin"), makevocab=True)

  # Chunk the data. This splits each of train.bin, val.bin and test.bin into smaller chunks, each containing e.g. 1000 examples, and saves them in finished_files/chunks
  chunk_all()

open progress bar
Successfully finished tokenizing to G:\master thesis\bigPatent_dataset_csv\train\cnn_stories_tokenized_train .

open progress bar
Successfully finished tokenizing to G:\master thesis\bigPatent_dataset_csv\train\cnn_stories_tokenized_test .

open progress bar
Successfully finished tokenizing to G:\master thesis\bigPatent_dataset_csv\train\cnn_stories_tokenized_val .

Writing story 0 of 1914; 0.00 percent done
Writing story 1000 of 1914; 52.25 percent done
Finished writing file G:\master thesis\bigPatent_dataset_csv\train\bp_finished_files\test.bin

Writing story 0 of 1914; 0.00 percent done
Writing story 1000 of 1914; 52.25 percent done
Finished writing file G:\master thesis\bigPatent_dataset_csv\train\bp_finished_files\val.bin

Writing story 0 of 34443; 0.00 percent done
Writing story 1000 of 34443; 2.90 percent done
Writing story 2000 of 34443; 5.81 percent done
Writing story 3000 of 34443; 8.71 percent done
Writing story 4000 of 34443; 11.61 percent done
Writing sto